## 시설물 점검진단계획 목록

In [39]:
from common import commonFunc as cf
import pandas as pd
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "시설물 점검진단계획 목록"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/시설물 점검진단계획 목록/selectIoFmChckDinsPlnList.csv'

In [40]:
# 개별 시설물 점검진단계획목록 수
g_totalCount = 0

In [41]:
##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
SERVICENAME

'selectIoFmChckDinsPlnList'

In [42]:
# REQPARAM = targetData["요청변수"].values[0]
# REQPARAM = REQPARAM.split(",")
# PRIMARYKEY = targetData["기본키"].values[0]
# REQPARAM

In [43]:
JSONKEY = "detail1"
DUMMY = 0

In [44]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/시설물 목록/selectIoFmMngList.csv", encoding="ms949")

In [45]:
dfCombi = imsiDf.loc[:,["fcno"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]
dfCombiLen

7913

## 시설물 점검진단계획 목록 전용 스크랩 함수

In [93]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata == "NODATA_ERROR"):
                        print("{} page is empty".format(i))
                        break
                    elif( jsondata == "DB_ERROR"):
                        print("DB ERROR")
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 detailList1~5까지 봐야 한다.
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)
                # fcno를 키로 기본정보와 시설물정보를 merge하기 위함
                print("fcno ",emptyPd['fcno'].values[0])
                fcno = emptyPd['fcno'].values[0]
                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount == 0:
                        g_totalCount = int(response.json()["response"]["body"]["totalCount"])
                    jsondata = response.json()["response"]["body"]["items"]
                    if( jsondata == []):
                        print("items is empty")
                    else:
#                         jsondata["index"]=[0]
                        rowData = pd.DataFrame(jsondata)
                        
                        rowData['fcno'] = fcno # 시설물 정보에 fcno칼럼을 수동으로 추가 (머지키로 활용하기 위함)
                        #emptyPd = pd.concat([emptyPd,rowData],axis=1).reindex(emptyPd.index)
                        emptyPd = pd.merge(left = emptyPd , right = rowData, on = "fcno")
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [100]:

numOfRows = 1000 # 수행 성능향상을 위해 100개씩 요청하는 것으로 수정해야 함 10 -> 1000, BASEPARAM에도 추가해야 함
PAGEYN=0

In [101]:
resultDfMerged = pd.DataFrame()
resultDf = pd.DataFrame()

for i in range(0,dfCombiLen):
    fcnoValue = dfCombi.loc[i].fcno
    pageNo = 1
    global g_totalCount
    g_totalCount = 0
    print("i=",i," fcnoValue= ",fcnoValue)
    while True:
        BASEPARAM={"serviceKey":APIKEY,"pageNo": pageNo, 'fcno':fcnoValue, 'numOfRows':numOfRows, "type":"json"}
        resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
        if resultDf.empty: # 정상 데이터가 없는 경우
            break
        else:
            resultDfMerged = resultDfMerged.append(resultDf) 
            g_totalCount -= numOfRows
            print("g_totalCount : ", g_totalCount, " numOfRows : ", numOfRows)
            if(g_totalCount <= 0):
                break
            pageNo += 1
        

        

i= 0  fcnoValue=  017F2520A338CB0F98899109C3F1E127
1 page scraping start
fcno  017F2520A338CB0F98899109C3F1E127
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 1  fcnoValue=  017F25212C92C63041C749D8850C27F0
1 page scraping start
fcno  017F25212C92C63041C749D8850C27F0
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 2  fcnoValue=  017F2521A0A3E80974D725F1B6DB68EF
1 page scraping start
fcno  017F2521A0A3E80974D725F1B6DB68EF
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 3  fcnoValue=  017F25224310A850FB962985F3EC1D60
1 page scraping start
fcno  017F25224310A850FB962985F3EC1D60
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 4  fcnoValue=  017F2524A82A3053A24976923A690EA9
1 page scraping start
fcno  017F2524A82A3053A24976923A690EA9
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 5  fcnoValue=  017F252518748A0FACF1C149F048DDE7
1 page scraping start
fcno  017F252518748A0FACF1C149F0

KeyboardInterrupt: 

In [96]:
resultDfMerged.shape

(166155, 26)

In [97]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

시설물 점검진단계획 목록 save compled


In [98]:
resultDfMerged.shape

(166155, 26)